In [82]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

import re

import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from automaton import Automaton, evaluate
from model import PaddedEmbeddingModel, RecurrentEmbeddingModel
from util import equalp, plot_schedule

Load the training data sets.

In [49]:
sets = [1,2,3,4,5,6]
train_files = [f"data/train_{i}.pkl" for i in sets]
print("\n".join(train_files))

data/train_1.pkl
data/train_2.pkl
data/train_3.pkl
data/train_4.pkl
data/train_5.pkl
data/train_6.pkl


Define the main training loop.

In [72]:
def train(model, data_train):
    print("\nextracting (state,next_lane) pairs by replaying optimal lane ordering")
    states, actions = [], []
    for _, row in tqdm(data_train.iterrows(), total=data_train.shape[0], leave=False):
        instance, schedule, eta = row[['instance', 'y_opt', 'y_opt_eta']]
        eta = iter(eta)
        automaton = Automaton(instance)
        while not automaton.done:
            lane = next(eta)
            state = model.state_transform(automaton)
            action = model.inverse_action_transform(automaton, lane)
            automaton.step(lane)
    
            states.append(state)
            actions.append(action)
    
    train_set = TensorDataset(torch.vstack(states), torch.vstack(actions))
    
    epochs = 5
    batch_size = 10
    data_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    
    learning_rate = 1e-3
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    print("\ntraining model\n")
    model.train()
    for i in range(epochs):
        loss_total = 0
        print(f'epoch: {i}')
        for s, a in tqdm(data_loader, leave=False):
            optimizer.zero_grad()
            logit = model(s)
            loss = F.binary_cross_entropy_with_logits(logit, a)
            loss.backward()
            loss_total += loss
            optimizer.step()
        print(f"loss: {loss_total.item()}")
        
    return model

In [78]:
for train_file in train_files:
    model = PaddedEmbeddingModel(lanes=2, horizon=5).cuda()
    data = pd.read_pickle(train_file)
    print(f"file={train_file}")

    model = train(model, data)

    model_file = train_file.rsplit(".", 1)[0] + "_" + model.__class__.__name__ + ".pt"
    torch.save(model, model_file)

file=data/train_1.pkl

extracting (state,next_lane) pairs by replaying optimal lane ordering


  0%|          | 0/1000 [00:00<?, ?it/s]


training model

epoch: 0


  0%|          | 0/2000 [00:00<?, ?it/s]

loss: 428.26263427734375
epoch: 1


...

loss: 385.509033203125
epoch: 3


  0%|          | 0/3000 [00:00<?, ?it/s]

loss: 374.67022705078125
epoch: 4


  0%|          | 0/3000 [00:00<?, ?it/s]

loss: 365.861083984375


Evaluate the trained models on the test sets.

In [101]:
test_files = [f"data/test_{i}.pkl" for i in sets]
for test_file in test_files:
    i = int(re.findall(r'\d+', test_file)[0])
    model_file = f"data/train_{i}" + "_" + model.__class__.__name__ + ".pt"
    model = torch.load(model_file, weights_only=False)
    model.eval()
    
    data_test = pd.read_pickle(test_file)
    print(f"file={test_file}")

    trained_heuristic = lambda automaton: \
               model.action_transform(automaton, model(model.state_transform(automaton)))

    def eval(row):
        instance = row['instance']
        schedule = evaluate(instance, trained_heuristic)
        return schedule['obj']

    data_test['neural_obj'] = data_test.progress_apply(eval, axis=1, result_type='expand')
    data_test.to_pickle(test_file)

file=data/test_1.pkl


  0%|          | 0/100 [00:00<?, ?it/s]

file=data/test_2.pkl


  0%|          | 0/100 [00:00<?, ?it/s]

file=data/test_3.pkl


...

  0%|          | 0/100 [00:00<?, ?it/s]

file=data/test_5.pkl


  0%|          | 0/100 [00:00<?, ?it/s]

file=data/test_6.pkl


  0%|          | 0/100 [00:00<?, ?it/s]

Save the results to `results.csv` for report.

In [116]:
total = pd.DataFrame()
for test_file in test_files:
    i = int(re.findall(r'\d+', test_file)[0])
    data_set = pd.read_pickle(test_file)
    data_set['set_id'] = i
    total = pd.concat([total, data_set])

total = total[['set_id', 'opt_obj', 'threshold_obj', 'neural_obj', 'running_time_0', 'running_time_1', 'running_time_2']]
total.to_csv('../report/data/results.csv', index=False)